In [ ]:
import numpy as np 
import pandas as pd
from pandas.plotting import lag_plot
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Solve Yfinance Issue
from curl_cffi import requests

TICKER = "SPY"

INTERVAL = "1d"
if INTERVAL == "1h":
    PERIOD = "730d"
else: 
    PERIOD = "max"

SHIFT = 1 
LOOKBACK = 10000

STRATEGY = ["Volume"]

SESSION = requests.Session(impersonate="chrome")

def get_data(ticker = TICKER, session = SESSION, interval = INTERVAL, lookback = LOOKBACK):

    df = yf.download(ticker, session = session, interval = interval, period = PERIOD)
    df.columns = df.columns.get_level_values(0)
    df = df.reset_index(drop=True)

    return df.iloc[-lookback:, :]

# Add Features
# Add Target 
def add_target(df, shift = SHIFT):

    df[f"Close + {shift}"] = df["Close"].shift(-shift)
    df["Target"] = df[f"Close + {shift}"] > df["Close"]

    return df

# --- (5) Generate Regression Output ---
def generate_regression_output(df, features=STRATEGY, target="Target"):

    subset = df[features + [target]].dropna()
    print(subset.columns)
    X = subset[features] 
    y = subset[target] 
    X_with_const = sm.add_constant(X)
    
    model = sm.Logit(y, X_with_const).fit() 
    print(model.summary())

    return df


# Validate Regression Output

def main():

    df = get_data()
    df = add_target(df)
    df = generate_regression_output(df)

    return df 

df = main()
df


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Index(['Volume', 'Target'], dtype='object', name='Price')
Optimization terminated successfully.
         Current function value: 0.689715
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                 Target   No. Observations:                 8278
Model:                          Logit   Df Residuals:                     8276
Method:                           MLE   Df Model:                            1
Date:                Tue, 16 Dec 2025   Pseudo R-squ.:               0.0001076
Time:                        18:23:22   Log-Likelihood:                -5709.5
converged:                       True   LL-Null:                       -5710.1
Covariance Type:            nonrobust   LLR p-value:                    0.2677
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1415      0.030      4.705      0

Price,Close,High,Low,Open,Volume,Close + 1,Target
0,24.313025,24.330317,24.209271,24.330317,1003200,24.485968,True
1,24.485968,24.485968,24.330336,24.330336,480500,24.537840,True
2,24.537840,24.555132,24.416793,24.468671,201300,24.797234,True
3,24.797234,24.814526,24.555141,24.572433,529400,24.900995,True
4,24.900995,24.952872,24.607025,24.883703,531500,24.883678,False
...,...,...,...,...,...,...,...
8273,687.570007,688.969971,681.309998,682.559998,85671300,689.169983,True
8274,689.169983,689.250000,682.169983,685.140015,86173700,681.760010,False
8275,681.760010,688.880005,679.169983,688.169983,113160300,680.729980,False
8276,680.729980,685.760010,679.250000,685.739990,90667700,678.419983,False
